## histwords词向量应用
1. histwords使用  https://nlp.stanford.edu/projects/histwords/
3. !git clone https://gitee.com/nju_cz/histwords 代码下载
4. 模型下载 http://snap.stanford.edu/historical_embeddings/ 路径下下载相应的模型
5. 下载后解压，放到histwords文件夹中的embeddings文件夹中

In [ ]:
import os
os.chdir('histwords')  # 切换到项目根目录
!pwd

In [ ]:
from representations.sequentialembedding import SequentialEmbedding

"""
Example showing how to load a series of historical embeddings and compute similarities over time.
Warning that loading all the embeddings into main memory can take a lot of RAM
"""

if __name__ == "__main__":
    fiction_embeddings = SequentialEmbedding.load("embeddings/eng-fiction-all_sgns", range(1950, 2000, 10))
    time_sims = fiction_embeddings.get_time_sims("lesbian", "gay")   
    print("Similarity between gay and lesbian drastically increases from 1950s to the 1990s:")
    for year, sim in time_sims.items():
        print("{year:d}, cosine similarity={sim:0.2f}".format(year=year,sim=sim))


In [ ]:
if __name__ == "__main__":
    fiction_embeddings = SequentialEmbedding.load("embeddings/chi-sim-all_sgns", range(1950, 2000, 10))
    time_sims = fiction_embeddings.get_time_sims("农民", "贫穷")   
    print("Similarity between 农民 and 贫穷 drastically increases from 1950s to the 1990s:")
    for year, sim in time_sims.items():
        print("{year:d}, cosine similarity={sim:0.2f}".format(year=year,sim=sim))

In [ ]:
if __name__ == "__main__":
    fiction_embeddings = SequentialEmbedding.load("embeddings/chi-sim-all_sgns", range(1950, 2000, 10))
    time_sims = fiction_embeddings.get_time_sims("男", "强壮")   
    print("Similarity between 男 and 强壮 drastically increases from 1950s to the 1990s:")
    for year, sim in time_sims.items():
        print("{year:d}, cosine similarity={sim:0.2f}".format(year=year,sim=sim))
    
    time_sims = fiction_embeddings.get_time_sims("女", "强壮")   
    print("Similarity between 女 and 强壮 drastically increases from 1950s to the 1990s:")
    for year, sim in time_sims.items():
        print("{year:d}, cosine similarity={sim:0.2f}".format(year=year,sim=sim))

In [ ]:
#复现性别职业刻板印象的计算
occupation_words = [
    "janitor", "statistician", "midwife", "bailiff", "auctioneer", "photographer", "geologist", 
    "shoemaker", "athlete", "cashier", "dancer", "housekeeper", "accountant", "physicist", 
    "gardener", "dentist", "weaver", "blacksmith", "psychologist", "supervisor", "mathematician", 
    "surveyor", "tailor", "designer", "economist", "mechanic", "laborer", "postmaster", "broker", 
    "chemist", "librarian", "attendant", "clerical", "musician", "porter", "scientist", "carpenter", 
    "sailor", "instructor", "sheriff", "pilot", "inspector", "mason", "baker", "administrator", 
    "architect", "collector", "operator", "surgeon", "driver", "painter", "conductor", "nurse", 
    "cook", "engineer", "retired", "sales", "lawyer", "clergy", "physician", "farmer", "clerk", 
    "manager", "guard", "artist", "smith", "official", "police", "doctor", "professor", "student", 
    "judge", "teacher", "author", "secretary", "soldier"
]

man_words = [
    "he", "son", "his", "him", "father", "man", "boy", "himself", "male", "brother", "sons", "fathers",
    "men", "boys", "males", "brothers", "uncle", "uncles", "nephew", "nephews"
]

woman_words = [
    "she", "daughter", "hers", "her", "mother", "woman", "girl", "herself", "female", "sister", 
    "daughters", "mothers", "women", "girls", "femen", "sisters", "aunt", "aunts", "niece", "nieces"
]

In [ ]:
import pandas as pd
df_man = pd.DataFrame({'年份': list(range(1900, 2000, 10))})
fiction_embeddings = SequentialEmbedding.load("embeddings/eng-all", range(1900, 2000, 10))
for man_word in man_words:
    for occupation_word in occupation_words:
        df_man[man_word + '-' +occupation_word] = fiction_embeddings.get_time_sims(man_word, occupation_word).values()
df_man = df_man.replace(0, None)
df_man.set_index('年份',inplace=True)
df_man.mean(axis=1)

In [ ]:
df_woman = pd.DataFrame({'年份': list(range(1900, 2000, 10))})
fiction_embeddings = SequentialEmbedding.load("embeddings/eng-all", range(1900, 2000, 10))
for woman_word in woman_words:
    for occupation_word in occupation_words:
        df_woman[woman_word + '-' +occupation_word] = fiction_embeddings.get_time_sims(woman_word, occupation_word).values()
df_woman = df_woman.replace(0, None)
df_woman.set_index('年份',inplace=True)
df_woman.mean(axis=1)

In [ ]:
bias=pd.concat([df_man.mean(axis=1),df_woman.mean(axis=1)],axis=1)
bias['bias']= bias[1]-bias[0]
bias

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 5))
plt.plot(bias.index, bias['bias'], marker='o')

plt.title("Bias Over Time")
plt.xlabel("Year")
plt.ylabel("Bias Score")
plt.grid(True)
plt.tight_layout()
plt.show()

## Doc2vec 
Doc2Vec（Document to Vector） 是一种将整段文本（如句子、段落、文章）转换为固定长度向量的模型，由 Mikolov 等人在 Word2Vec 的基础上提出。它能捕捉文本中的语义信息，使得我们可以使用向量数学来比较文本之间的相似度。

Doc2Vec 是 Word2Vec 的扩展，目标是将每个文档映射为一个向量，不仅仅是对词向量求平均。

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [ ]:
def protory_wash(content):
    stoplist=[line.strip() for line in open('中文stoplist.txt').readlines()]
    result=[token for token in list(content) if token not in stoplist]
    return result

#文本预处理
data = pd.read_excel('唐诗三百首.xlsx') 
texts = data['诗词内容'].dropna().tolist() 
titles = data['题目'].dropna().tolist() 
cleaned_texts = [protory_wash(doc) for doc in texts]

tagged_data = [TaggedDocument(words=words, tags=[str(i)]) for i, words in enumerate(cleaned_texts)] # 将每个句子转换为 TaggedDocument 对象

# 训练 Doc2Vec 模型
model = Doc2Vec(vector_size=300, window=5, min_count=1, workers=4, epochs=100)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# 推理每首诗的向量
vectors = [model.infer_vector(doc) for doc in cleaned_texts]


In [ ]:
# 计算相似度矩阵
similarity_matrix = cosine_similarity(vectors)
# 示例：比较第0首和第1首诗的相似度
index1 = 0
index2 = 1
sim_score = similarity_matrix[index1][index2]

print(f"《{titles[index1]}》 与 《{titles[index2]}》 的相似度为：{sim_score:.4f}")

In [ ]:
def find_most_similar_poem(index):
    sims = list(enumerate(similarity_matrix[index]))
    # 排除自身（自己与自己相似度为1）
    sims = [(i, score) for i, score in sims if i != index]
    # 按相似度降序排序
    most_similar = max(sims, key=lambda x: x[1])
    similar_index, similarity_score = most_similar

    print(f"与《{titles[index]}》最相似的是《{titles[similar_index]}》，相似度为：{similarity_score:.4f}")
    print("\n原诗内容：")
    print(texts[index])
    print("\n相似诗内容：")
    print(texts[similar_index])

# 示例：查找与第0首诗最相似的另一首诗
find_most_similar_poem(100)